In [1]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D, Activation

import tensorflow as tf

import params
import dataset

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

/home/valentin/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def BuildModel():
    inp = Input(shape=(params.GetImageSize(), params.GetImageSize(), params.GetChannelsNum()))

    conv_1_1 = Conv2D(32, (3, 3), padding='same')(inp)
    conv_1_1 = Activation('relu')(conv_1_1)
    conv_1_2 = Conv2D(32, (3, 3), padding='same')(conv_1_1)
    conv_1_2 = Activation('relu')(conv_1_2)
    pool_1 = MaxPooling2D(2)(conv_1_2)

    conv_2_1 = Conv2D(64, (3, 3), padding='same')(pool_1)
    conv_2_1 = Activation('relu')(conv_2_1)
    conv_2_2 = Conv2D(64, (3, 3), padding='same')(conv_2_1)
    conv_2_2 = Activation('relu')(conv_2_2)
    pool_2 = MaxPooling2D(2)(conv_2_2)

    conv_3_1 = Conv2D(128, (3, 3), padding='same')(pool_2)
    conv_3_1 = Activation('relu')(conv_3_1)
    conv_3_2 = Conv2D(128, (3, 3), padding='same')(conv_3_1)
    conv_3_2 = Activation('relu')(conv_3_2)
    pool_3 = MaxPooling2D(2)(conv_3_2)

    up_1 = UpSampling2D(2, interpolation='bilinear')(pool_3)
    conc_1 = Concatenate()([conv_3_2, up_1])
    conv_up_1_1 = Conv2D(256, (3, 3), padding='same')(up_1)
    conv_up_1_1 = Activation('relu')(conv_up_1_1)
    conv_up_1_2 = Conv2D(256, (3, 3), padding='same')(conv_up_1_1)
    conv_up_1_2 = Activation('relu')(conv_up_1_2)

    up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)
    conc_2 = Concatenate()([conv_2_2, up_2])
    conv_up_2_1 = Conv2D(128, (3, 3), padding='same')(up_2)
    conv_up_2_1 = Activation('relu')(conv_up_2_1)
    conv_up_2_2 = Conv2D(128, (3, 3), padding='same')(conv_up_2_1)
    conv_up_2_2 = Activation('relu')(conv_up_2_2)

    up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)
    conc_3 = Concatenate()([conv_1_2, up_3])
    conv_up_3_1 = Conv2D(64, (3, 3), padding='same')(up_3)
    conv_up_3_1 = Activation('relu')(conv_up_3_1)
    conv_up_3_2 = Conv2D(4, (3, 3), padding='same')(conv_up_3_1)
    result = Activation('sigmoid')(conv_up_3_2)
    
    return Model(inputs=inp, outputs=result)

model = BuildModel()

In [3]:
best_w = keras.callbacks.ModelCheckpoint('unet_best.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=True,
                                save_weights_only=True,
                                mode='auto',
                                period=1)

last_w = keras.callbacks.ModelCheckpoint('unet_last.h5',
                                monitor='val_loss',
                                verbose=0,
                                save_best_only=False,
                                save_weights_only=True,
                                mode='auto',
                                period=1)


callbacks = [best_w, last_w]



adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)


model.compile(adam, 'categorical_crossentropy')

In [4]:
data = dataset.read_train_validation_big_sets(
        train_path = os.path.join(params.training_data_path),
        validation_path = os.path.join(params.validation_data_path),
        image_size = params.GetImageSize())

print("Complete reading input data. Will Now print a snippet of it")
print("Number of files in Training-set:\t\t{}".format(data.train.num_examples))
print("Number of files in Validation-set:\t{}".format(data.valid.num_examples))

Complete reading input data. Will Now print a snippet of it
Number of files in Training-set:		200
Number of files in Validation-set:	40


In [5]:
img_df = data.train.image_paths
mask_df = data.train.masks_paths
batch_size = 26

index = 0
while index < 5:
    x_batch = []
    y_batch = []
    r_batch = []

    for i in range(batch_size):
        img_name = img_df[index]
        mask_name = mask_df[index]

#             print(index, img_name, mask_name)

        index = (index + 1) % len(img_df)

        img = cv2.imread(img_name, cv2.IMREAD_COLOR)
        mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)

#             img, mask

        img = cv2.resize(img, (params.GetImageSize(), params.GetImageSize()))
        mask = cv2.resize(mask, (params.GetImageSize(), params.GetImageSize()))            

        height, width = mask.shape
        masks = np.zeros((height, width, 4))
        for i in range(masks.shape[2]):
            masks[:, :, i] = (mask == (i+1))

        #moving the channel:
        mask_train = np.moveaxis(masks,-1,1)

        x_batch += [img]
        y_batch.append(mask_train)
        r_batch += [mask]

    x_batch = np.array(x_batch) / 255.
    y_batch = np.array(y_batch) / 255.
    r_batch = np.array(r_batch) / 255.

In [6]:
x_batch.shape,y_batch.shape,r_batch.shape

((26, 256, 256, 3), (26, 256, 4, 256), (26, 256, 256))

In [7]:
def keras_generator(img_df, mask_df, batch_size):
    index = 0
    while True:
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            img_name = img_df[index]
            mask_name = mask_df[index]
            
#             print(index, img_name, mask_name)
            
            index = (index + 1) % len(img_df)
            
            img = cv2.imread(img_name, cv2.IMREAD_COLOR)
            mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)
            
#             img, mask
            
            img = cv2.resize(img, (params.GetImageSize(), params.GetImageSize()))
            mask = cv2.resize(mask, (params.GetImageSize(), params.GetImageSize()))            
            
            height, width = mask.shape
            masks = np.zeros((height, width, 4))
            for i in range(masks.shape[2]):
                masks[:, :, i] = (mask == (i+1))
            
#             masks = np.moveaxis(masks,-1,1)
            
            x_batch += [img]
            y_batch.append(masks)

        x_batch = np.array(x_batch) / 255.
        y_batch = np.array(y_batch) / 1.
        
#         print(x_batch.shape,y_batch.shape)

        yield x_batch, y_batch

In [ ]:
batch_size = 16
model.fit_generator(keras_generator(data.train.image_paths, data.train.masks_paths, batch_size),
              steps_per_epoch=100,
              epochs=3,
              verbose=1,
              callbacks=callbacks,
              validation_data=keras_generator(data.valid.image_paths, data.valid.masks_paths, batch_size),
              validation_steps=50,
              class_weight=None,
              max_queue_size=10,
              workers=1,
              use_multiprocessing=True,
              shuffle=True,
              initial_epoch=0)

Epoch 1/3
 10/100 [==>...........................] - ETA: 36:38 - loss: 1.3577

In [ ]:
from keras.models import load_model

model1 = BuildModel()
model1.load_weights('unet_best.h5')
# model1 = load_model('unet_best.h5')

In [ ]:
for x, y in keras_generator(data.valid.image_paths, data.valid.masks_paths, 16):
    break

In [ ]:
pred = model1.predict(x)

In [ ]:
pred[0].shape

In [ ]:
sample_index = 0
keras.backend.eval(tf.keras.backend.argmax(pred[sample_index], 2)).shape

In [ ]:
def getClassesMap(pred):
    res = np.zeros((pred.shape[0], pred.shape[1]), np.uint8)
    for r in range(pred.shape[0]):
        for c in range(pred.shape[1]):
            pred_class_index = np.argmax(pred[r,c,:])
            res[r,c] = pred_class_index
    res.shape
    return res

In [ ]:
getClassesMap(pred[sample_index]).shape

In [ ]:
sample_index = 1
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(25, 25))
axes[0][0].imshow(x[sample_index])
im_0_1 = axes[0][1].imshow(pred[sample_index, ..., 0])
im_1_0 = axes[1][0].imshow(pred[sample_index, ..., 1])
im_1_1 = axes[1][1].imshow(pred[sample_index, ..., 2])
im_2_0 = axes[2][0].imshow(pred[sample_index, ..., 3])
# im_2_1 = axes[2][1].imshow(keras.backend.eval(tf.keras.backend.argmax(pred[sample_index], 2)).astype(np.float32))
# im_2_1 = axes[2][1].imshow(getClassesMap(pred[sample_index]))
im_2_1 = axes[2][1].imshow(pred[sample_index, 20:40, 0:5, 0])

plt.colorbar(im_0_1, ax=axes[0,1])
plt.colorbar(im_1_0, ax=axes[1,0])
plt.colorbar(im_1_1, ax=axes[1,1])
plt.colorbar(im_2_0, ax=axes[2,0])
plt.colorbar(im_2_1, ax=axes[2,1])

plt.show()

In [ ]:
np.max(pred[sample_index, 20:40, 0:5, 3])

In [ ]:
pred[sample_index][15,110,:]

In [ ]:
np.exp(pred[sample_index][15,110,:])/sum(np.exp(pred[sample_index][15,110,:]))

In [ ]:
a=pred[sample_index][15,110,:]
np.argmax(a)

In [ ]:
keras.backend.eval(tf.keras.backend.argmax(pred[sample_index], 2))

In [ ]:
y[sample_index][:,:,0].shape

In [ ]:
keras.backend.eval(tf.keras.backend.argmax(pred[sample_index], 2)) == (y[sample_index][:,:,1])

In [ ]:
sample_index = 1

ok_cnt = 0
errors = {
    0: [0,0,0,0],
    1: [0,0,0,0],
    2: [0,0,0,0],
    3: [0,0,0,0],
}
for r in range(y[sample_index].shape[0]):
    for c in range(y[sample_index].shape[1]):
        pred_class_index = np.argmax(pred[sample_index][r,c,:]) + 0
        real_class_index = np.argmax(y[sample_index][r,c,:])
#         print(r,c,pred_class_index, real_class_index)
        if pred_class_index == real_class_index:
            ok_cnt = ok_cnt + 1
        else:
            errors[real_class_index][pred_class_index] = errors[real_class_index][pred_class_index] + 1
        

In [ ]:
ok_cnt,errors